In [36]:
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit.quantum_info import SparsePauliOp, Pauli
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.units import DistanceUnit
import numpy as np
from qiskit_nature.second_q.mappers import JordanWignerMapper

In [37]:
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.742",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

molecule = driver.run()
mapper = JordanWignerMapper()
hamiltonian1 = mapper.map(molecule.hamiltonian.second_q_op())

In [38]:

cost_h=hamiltonian1

driver_scaling = 0.25
driver_h = SparsePauliOp.from_list([
    ("XIII", -1.0*driver_scaling),
    ("IXII", -1.0*driver_scaling),
    ("IIXI", -1.0*driver_scaling),
    ("IIIX", -1.0*driver_scaling),
])
print(driver_h)


from qiskit.quantum_info import SparsePauliOp

def build_commutator(op_a: SparsePauliOp, op_b: SparsePauliOp) -> SparsePauliOp:
    ab = op_a @ op_b
    ba = op_b @ op_a
    comm_pre= ab - ba
    comm = comm_pre*1j
    return comm.simplify()

comm_h=build_commutator(driver_h, cost_h)
comm_h = comm_h.simplify()


SparsePauliOp(['XIII', 'IXII', 'IIXI', 'IIIX'],
              coeffs=[-0.25+0.j, -0.25+0.j, -0.25+0.j, -0.25+0.j])


In [39]:
from qiskit_algorithms import TimeEvolutionProblem


In [40]:
from qiskit_algorithms import TrotterQRTE
trotter = TrotterQRTE()
def falqon_layer_2(state,beta_k, cost_h, driver_h, delta_t):
    
    problem1 = TimeEvolutionProblem(
    hamiltonian=cost_h,
    time=0.25,
    initial_state=state 
    )
    result1 = trotter.evolve(problem1)
    evolved_circuit1 = result1.evolved_state
    new_state = Statevector.from_instruction(evolved_circuit1)

# 步骤 2：用 cost_h 演化新态
    problem2 = TimeEvolutionProblem(
    hamiltonian=driver_h,
    time=beta_k*0.25,
    initial_state=new_state
    )
    result2 = trotter.evolve(problem2)
    evolved_circuit2 = result2.evolved_state
    final_state = Statevector.from_instruction(evolved_circuit2)
    return final_state


    

In [41]:
def main_loop(n_qubits, n_layers,beta_1, delta_t, cost_h, driver_h, comm_h):
    beta = [beta_1]
    energies = []
    state = Statevector.from_label("+" * n_qubits)

    for i in range(n_layers):
        middle_state=falqon_layer_2(state, beta[i], cost_h, driver_h, delta_t)
        next_beta = -1*middle_state.expectation_value(comm_h).real
        beta.append(next_beta)
        energy = middle_state.expectation_value(cost_h).real
        energies.append(energy)
        state = middle_state
        print(f"第{i + 1}步：能量 = {energy:.8f} Ha | β_{i} = {beta[i]:+.6f}")
        
             
        if i >= 1:
            if abs(energies[-1] - energies[-2]) < 1e-6:
                print(f"✅ 收敛于第 {i + 1} 层，能量变化小于 1e-6 Ha")
                break

    return energies

In [42]:
n_layers = 500
beta_1 = 0
delta_t = 0.1
res_energies= main_loop(
    n_qubits=4,
    n_layers=n_layers,
    beta_1=beta_1,
    delta_t=delta_t,
    cost_h=cost_h,
    driver_h=driver_h,
    comm_h=comm_h
)

第1步：能量 = -0.76579447 Ha | β_0 = +0.000000
第2步：能量 = -0.76955641 Ha | β_1 = +0.102928
第3步：能量 = -0.78274816 Ha | β_2 = +0.198984
第4步：能量 = -0.80678691 Ha | β_3 = +0.280885
第5步：能量 = -0.84013675 Ha | β_4 = +0.342231
第6步：能量 = -0.87901784 Ha | β_5 = +0.379158
第7步：能量 = -0.91892064 Ha | β_6 = +0.391586
第8步：能量 = -0.95609966 Ha | β_7 = +0.383223
第9步：能量 = -0.98833036 Ha | β_8 = +0.360180
第10步：能量 = -1.01483723 Ha | β_9 = +0.328983
第11步：能量 = -1.03578972 Ha | β_10 = +0.295000
第12步：能量 = -1.05178937 Ha | β_11 = +0.261757
第13步：能量 = -1.06354277 Ha | β_12 = +0.231004
第14步：能量 = -1.07172087 Ha | β_13 = +0.203155
第15步：能量 = -1.07693311 Ha | β_14 = +0.177786
第16步：能量 = -1.07974985 Ha | β_15 = +0.154051
第17步：能量 = -1.08073306 Ha | β_16 = +0.130992
第18步：能量 = -1.08045638 Ha | β_17 = +0.107739
第19步：能量 = -1.07950836 Ha | β_18 = +0.083625
第20步：能量 = -1.07847788 Ha | β_19 = +0.058255
第21步：能量 = -1.07792366 Ha | β_20 = +0.031535
第22步：能量 = -1.07833014 Ha | β_21 = +0.003696
第23步：能量 = -1.08005275 Ha | β_22 = -0.024675
第24步：能量